In [1]:
import minsearch
import json

In [2]:
with open('documents.json', 'rt') as data_file:
    docs_raw = json.load(data_file)

In [3]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(text_fields=['question',"text", "section"], 
               keyword_fields=["course"])

In [6]:
index.fit(documents)

In [7]:
q = 'The course has started, can i still enroll?'

In [8]:
boost ={'question':3.0,'section':0.5}

results = index.search(
    query = q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost, num_results=5
    
)

In [9]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [10]:
from openai import OpenAI

In [11]:
client = OpenAI()

In [12]:
response = client.chat.completions.create(model ='gpt-4o', messages =[{"role":"user", "content":q}])

In [13]:
response.choices[0].message.content

"The ability to enroll in a course after it has started generally depends on the policies of the institution offering the course. Here are a few steps you can take to find out if it's still possible:\n\n1. **Check the Course Website or Catalog:** Look for any specific deadlines or late registration policies.\n\n2. **Contact the Instructor:** Sometimes, instructors have the discretion to allow late enrollments and can provide you with the necessary guidance or permissions.\n\n3. **Contact the Registrar's Office or Academic Advisor:** They can give you detailed information about enrollment policies and help you navigate any necessary procedures.\n\n4. **Review Institutional Policies:** Some institutions have a grace period for adding or dropping courses with minimal penalties.\n\n5. **Consider Auditing:** If enrollment is closed, ask if you can audit the course, which means you can attend and participate without receiving a grade or formal credit.\n\nMake sure to act quickly, as the long

In [14]:
def search(query):
    boost ={'question':3.0,'section':0.5}

    results = index.search(
        query = query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost, num_results=5
        
    )
    return results

In [15]:
def build_prompt(query,search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT form the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
  
    
    QUESTION:{question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"section:{doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    prompt=prompt_template.format(question=query, context=context).strip()
    return prompt

In [16]:
def llm(prompt):
    response = client.chat.completions.create(model ='gpt-4o', messages =[{"role":"user", "content":prompt}])
    return response.choices[0].message.content
    

In [17]:
query = 'how do I run kafka?'

def rag(query):
    results = search(query)
    prompt = build_prompt(query,results)
    answer = llm(prompt)
    return answer

In [18]:
rag(query)

'To run Kafka in your environment, you can follow these steps based on the nature of your project:\n\n1. **Java Kafka Producer/Consumer/KStreams**:\n   - In the project directory, you can run the following command to start a producer, consumer, or KStreams:\n     ```sh\n     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n     ```\n\n2. **Python Kafka**:\n   - If you encounter a "Module ‘kafka’ not found" error while trying to run `producer.py`, you should create a virtual environment first and install the required packages:\n     ```sh\n     python -m venv env\n     source env/bin/activate\n     pip install -r ../requirements.txt\n     ```\n   - To activate the virtual environment each time you need it, use:\n     ```sh\n     source env/bin/activate\n     ```\n   - To deactivate the virtual environment, use:\n     ```sh\n     deactivate\n     ```\n   - Note: For Windows, to activate the virtual environment, you should use:\n     ```sh\n

In [19]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [20]:
from elasticsearch import Elasticsearch 

In [21]:
es_client = Elasticsearch('http://localhost:9200')

In [23]:
es_client.info()

ObjectApiResponse({'name': '9fc0d736f48c', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'QJo2Y0GwTdS1FAvmSKAyMA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [25]:
index_settings= {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [27]:
from tqdm.auto import tqdm

In [30]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████| 948/948 [00:20<00:00, 45.80it/s]


In [31]:
query = 'I just discovered the course, can I still join it?'

In [39]:
def search_elastic(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs=[]
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [41]:
def rag(query):
    results = search_elastic(query)
    prompt = build_prompt(query,results)
    answer = llm(prompt)
    return answer